In [2]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes==0.43.1
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q transformers[sentencepiece]
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")


2024-09-12 14:57:44.973801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/ematos/miniconda3/envs/gpu-env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [4]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/home/ematos/miniconda3/envs/gpu-env/lib/python3.11/site-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [6]:
filepath = '../files/texto01.txt'
text = open(filepath, encoding='utf-8').read()

In [7]:
ner_results = nlp(text)
print(ner_results)

[{'entity_group': 'PER', 'score': 0.5869531, 'word': 'Senhora Senhora Rivail', 'start': 4, 'end': 27}, {'entity_group': 'PER', 'score': 0.81715226, 'word': 'Boudet', 'start': 57, 'end': 63}, {'entity_group': 'LOC', 'score': 0.99497426, 'word': 'Château - du - Loir', 'start': 75, 'end': 90}, {'entity_group': 'LOC', 'score': 0.6078252, 'word': 'Sarthe', 'start': 92, 'end': 98}, {'entity_group': 'PER', 'score': 0.9522022, 'word': 'Amélie', 'start': 286, 'end': 292}, {'entity_group': 'PER', 'score': 0.87714845, 'word': 'Crispoul', 'start': 792, 'end': 800}, {'entity_group': 'PER', 'score': 0.87948895, 'word': 'Grandins', 'start': 896, 'end': 904}, {'entity_group': 'PER', 'score': 0.93088436, 'word': 'Leroux', 'start': 909, 'end': 915}, {'entity_group': 'PER', 'score': 0.9403838, 'word': 'Ferran', 'start': 1360, 'end': 1366}, {'entity_group': 'PER', 'score': 0.42519313, 'word': 'Ballet', 'start': 1461, 'end': 1467}, {'entity_group': 'PER', 'score': 0.95294714, 'word': 'Morin', 'start': 1829

In [11]:
import json
filepath = '../files/texto01_result.txt'
f = open(filepath, "w")
f.write(json.dumps(str(ner_results)))
f.close()

In [24]:
import mysql.connector as mariadb
import sys

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="kardec",
        password="",
        host="projetokardec.ufjf.br",
        port=33069,
        database="kardec_db"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()
# Select tagged sentences without a NLP label. NLP labels are set for non-informative sentences.
cur.execute("SELECT idSentence as id, text from ak_sentence where idlanguage=1")
#cur.execute("SELECT idSentence as id, text from ak_sentence where idlanguage=1 and (idSentence < 50)")
rows = cur.fetchall()
filepath = '../files/full_result.txt'
f = open(filepath, "w")
for row in rows:
#    print(row[0], row[1])
    ner_results = nlp(row[1])
    for result in ner_results:
        if (result['entity_group'] == 'PER') or (result['entity_group'] == 'LOC'):
            f.write(str(row[0]) + ',' + result['entity_group'] +  ',' + result['word']+ "\n")
f.close()